In [1]:
import numpy as np
import cv2
import pandas as pd
from tqdm.notebook import tqdm 
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import os

In [2]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
original_fake_paths = []

for dirname, _, filenames in tqdm(os.walk('1-million-fake-faces/')):
    for filename in filenames:
        original_fake_paths.append([os.path.join(dirname, filename), filename])

In [4]:
save_dir = 'tmp/fake/'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [5]:
fake_paths = [save_dir + filename for _, filename in original_fake_paths]

In [6]:
for path, filename in tqdm(original_fake_paths):
    img = cv2.imread(path)
    img = cv2.resize(img, (224, 224))
    cv2.imwrite(os.path.join(save_dir, filename), img)

In [7]:
train_fake_paths, test_fake_paths = train_test_split(fake_paths, test_size=20000, random_state=2019)

fake_train_df = pd.DataFrame(train_fake_paths, columns=['filename'])
fake_train_df['class'] = 'FAKE'

fake_test_df = pd.DataFrame(test_fake_paths, columns=['filename'])
fake_test_df['class'] = 'FAKE'

In [8]:
real_dir = 'celeba-dataset/img_align_celeba/img_align_celeba/'
eval_partition = pd.read_csv('celeba-dataset/list_eval_partition.csv')

eval_partition['filename'] = eval_partition.image_id.apply(lambda st: real_dir + st)
eval_partition['class'] = 'REAL'

In [9]:
real_train_df = eval_partition.query('partition in [0, 1]')[['filename', 'class']]
real_test_df = eval_partition.query('partition == 2')[['filename', 'class']]

In [10]:
train_df = pd.concat([real_train_df, fake_train_df])
test_df = pd.concat([real_test_df, fake_test_df])

In [11]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_dataframe(
    train_df,
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_dataframe(
    train_df,
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary',
    subset='validation'
)

Found 258110 validated image filenames belonging to 2 classes.
Found 64527 validated image filenames belonging to 2 classes.


In [12]:
datagen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(
    test_df,
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

Found 39962 validated image filenames belonging to 2 classes.


In [14]:
densenet = DenseNet121(
    weights='densenet-keras/DenseNet-BC-121-32-no-top.h5',
    include_top=False,
    input_shape=(224,224,3)
)

for layer in densenet.layers:
    layer.trainable = False

In [15]:
def build_model(densenet):
    model = Sequential()
    model.add(densenet)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(lr=0.0005),
        metrics=['accuracy']
    )
    
    return model

In [16]:
model = build_model(densenet)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 7, 7, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               262400    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0

In [18]:
checkpoint = ModelCheckpoint('model.h5', save_best_only=True)

train_history_step1 = model.fit_generator(
    train_gen,
    validation_data=val_gen,
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    callbacks=[checkpoint],
    epochs=3
)

Train for 4033 steps, validate for 1009 steps
Epoch 1/3
4033/4033 [==============================] - 1262s 313ms/step - loss: 0.0036 - accuracy: 0.9989 - val_loss: 0.0326 - val_accuracy: 0.9892
Epoch 2/3
4033/4033 [==============================] - 1037s 257ms/step - loss: 0.0017 - accuracy: 0.9994 - val_loss: 0.0150 - val_accuracy: 0.9953
Epoch 3/3
4033/4033 [==============================] - 963s 239ms/step - loss: 0.0014 - accuracy: 0.9995


In [19]:
model.load_weights('model.h5')
for layer in model.layers:
    layer.trainable = True

train_history_step2 = model.fit_generator(
    train_gen,
    validation_data=val_gen,
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    callbacks=[checkpoint],
    epochs=3
)

Train for 4033 steps, validate for 1009 steps
Epoch 1/3
4033/4033 [==============================] - 1004s 249ms/step - loss: 0.0016 - accuracy: 0.9995 - val_loss: 0.0010 - val_accuracy: 0.9997
Epoch 2/3
4033/4033 [==============================] - 1023s 254ms/step - loss: 0.0014 - accuracy: 0.9996 - val_loss: 0.0026 - val_accuracy: 0.9993
Epoch 3/3
4033/4033 [==============================] - 1023s 254ms/step - loss: 0.0012 - accuracy: 0.9996 - val_loss: 0.0061 - val_accuracy: 0.9983


In [21]:

pd.DataFrame(train_history_step2.history).to_csv('history2.csv')